# sql-ex.ru 

Схема БД состоит из четырех таблиц:

* **Product**(maker, model, type)

* **PC**(code, model, speed, ram, hd, cd, price)

* **Laptop**(code, model, speed, ram, hd, price, screen)

* **Printer**(code, model, color, type, price)

Таблица **Product** представляет производителя (maker), номер модели (model) и тип ('PC' - ПК, 'Laptop' - ПК-блокнот или 'Printer' - принтер). Предполагается, что номера моделей в таблице Product уникальны для всех производителей и типов продуктов. 

<br>В таблице **PC** для каждого ПК, однозначно определяемого уникальным кодом – code, указаны модель – model (внешний ключ к таблице Product), скорость - speed (процессора в мегагерцах), объем памяти - ram (в мегабайтах), размер диска - hd (в гигабайтах), скорость считывающего устройства - cd (например, '4x') и цена - price (в долларах). 

<br>Таблица **Laptop** аналогична таблице РС за исключением того, что вместо скорости CD содержит размер экрана -screen (в дюймах). 

<br>В таблице **Printer** для каждой модели принтера указывается, является ли он цветным - color ('y', если цветной), тип принтера - type (лазерный – 'Laser', струйный – 'Jet' или матричный – 'Matrix') и цена - price.


**Cлишком простые задачи, а также задачи, не демонстрирующие новый синтаксис, в этот файл я не записываю**
****
**6. Для каждого производителя, выпускающего ПК-блокноты c объёмом жесткого диска не менее 10 Гбайт, найти скорости таких ПК-блокнотов. Вывод: производитель, скорость.**

```sql
SELECT DISTINCT maker, speed
FROM product p JOIN laptop l 
ON p.model=l.model
WHERE hd>=10
```
****
**Задание: 7 (Serge I: 2002-11-02)
Найдите номера моделей и цены всех имеющихся в продаже продуктов (любого типа) производителя B (латинская буква).**
```sql
SELECT * 
FROM ((SELECT model, price 
   FROM PC)
   UNION
   (SELECT model, price 
   FROM Laptop)
   UNION
   (SELECT model, price 
   FROM Printer)) AS p2
WHERE p2.model IN (SELECT model 
 FROM Product 
 WHERE maker = 'B')
```

<br> 3 WHERE или один WITH ? При больших данных, возможно, WITH быстрее
<br> (это предположение, я не знаю точно)
****
**Задание: 8 (Serge I: 2003-02-03)
Найдите производителя, выпускающего ПК, но не ПК-блокноты.**
```sql
SELECT maker
FROM product 
WHERE type='PC'
EXCEPT 
SELECT maker
FROM product 
WHERE type='Laptop'
```
****
**Задание: 10 (Serge I: 2002-09-23)
Найдите модели принтеров, имеющих самую высокую цену. Вывести: model, price**
```sql
SELECT model, price
FROM printer
WHERE price=(SELECT MAX(price) FROM printer)
```
****
**Задание: 15 (Serge I: 2003-02-03)
Найдите размеры жестких дисков, совпадающих у двух и более PC. Вывести: HD**
```sql
SELECT hd
FROM pc
GROUP BY hd
HAVING COUNT(*)>1
```
****
**Задание: 16 (Serge I: 2003-02-03)
Найдите пары моделей PC, имеющих одинаковые скорость и RAM. В результате каждая пара указывается только один раз, т.е. (i,j), но не (j,i), Порядок вывода: модель с большим номером, модель с меньшим номером, скорость и RAM.**
```sql
SELECT DISTINCT a.model, b.model, a.speed, a.ram
FROM pc a, pc b
WHERE a.speed=b.speed AND a.ram=b.ram 
AND a.model>b.model
```
****
**Задание: 18 (Serge I: 2003-02-03)
Найдите производителей самых дешевых цветных принтеров. Вывести: maker, price**
```sql
SELECT DISTINCT maker, price
FROM product p JOIN printer pr
ON p.model=pr.model
WHERE color='Y' AND price=(SELECT MIN(price) FROM printer WHERE color='Y')
```
****
**Задание: 20 (Serge I: 2003-02-13)
Найдите производителей, выпускающих по меньшей мере три различных модели ПК. Вывести: Maker, число моделей ПК.**
```sql
SELECT maker, COUNT(model) AS Count_Model
FROM product 
WHERE type='PC'
GROUP BY maker
HAVING COUNT(model)>=3
```
****
**Задание: 23 (Serge I: 2003-02-14)
Найдите производителей, которые производили бы как ПК
со скоростью не менее 750 МГц, так и ПК-блокноты со скоростью не менее 750 МГц.
Вывести: Maker**
```sql
SELECT maker
FROM product p JOIN pc
ON p.model=pc.model
WHERE speed>=750
INTERSECT
SELECT maker
FROM product p JOIN laptop l
ON p.model=l.model
WHERE speed>=750
```
****
**Задание: 24 (Serge I: 2003-02-03)
Перечислите номера моделей любых типов, имеющих самую высокую цену по всей имеющейся в базе данных продукции.**
```sql
WITH p2 AS
(SELECT model, price  
FROM laptop  
UNION
(SELECT model, price 
FROM pc)
UNION
(SELECT model, price 
FROM printer))

SELECT p.model
FROM product p JOIN p2
ON p.model=p2.model
WHERE price=(SELECT MAX(price) FROM p2)
```
****
**Задание: 25 (Serge I: 2003-02-14)
Найдите производителей принтеров, которые производят ПК с наименьшим объемом RAM и с самым быстрым процессором среди всех ПК, имеющих наименьший объем RAM. Вывести: Make**
```sql
SELECT maker 
FROM product p JOIN pc
ON p.model=pc.model
WHERE ram=(SELECT MIN(ram) FROM pc)
AND speed=(SELECT MAX(speed) FROM pc 
    WHERE ram=(SELECT MIN(ram) FROM pc))
INTERSECT
SELECT maker 
FROM product p
WHERE type='Printer'
```
****
**Задание: 26 (Serge I: 2003-02-14)
Найдите среднюю цену ПК и ПК-блокнотов, выпущенных производителем A (латинская буква). Вывести: одна общая средняя цена.**
```sql
WITH b AS
(SELECT price, model FROM pc
UNION ALL
SELECT price, model FROM laptop)

SELECT avg(price) FROM
Product p JOIN
b ON p.model=b.model
WHERE maker='A'
```
****
**Задание: 27 (Serge I: 2003-02-03)
Найдите средний размер диска ПК каждого из тех производителей, которые выпускают и принтеры. Вывести: maker, средний размер HD.**
<br> задачка простая, удалить?
```sql
SELECT maker, AVG(hd)
FROM pc
JOIN product p
ON p.model=pc.model
WHERE maker IN (SELECT maker FROM product WHERE type='Printer')
GROUP BY maker
```
Фирма имеет несколько пунктов приема вторсырья. Каждый пункт получает деньги для их выдачи сдатчикам вторсырья. Сведения о получении денег на пунктах приема записываются в таблицу:
* **Income_o(point, date, inc)**
Первичным ключом является (point, date). При этом в столбец date записывается только дата (без времени), т.е. прием денег (inc) на каждом пункте производится не чаще одного раза в день. Сведения о выдаче денег сдатчикам вторсырья записываются в таблицу:
* **Outcome_o(point, date, out)**
В этой таблице также первичный ключ (point, date) гарантирует отчетность каждого пункта о выданных деньгах (out) не чаще одного раза в день.
В случае, когда приход и расход денег может фиксироваться несколько раз в день, используется другая схема с таблицами, имеющими первичный ключ code:
* **Income(code, point, date, inc)**
* **Outcome(code, point, date, out)**
Здесь также значения столбца date не содержат времени.
****
**Задание: 29 (Serge I: 2003-02-14)
В предположении, что приход и расход денег на каждом пункте приема фиксируется не чаще одного раза в день [т.е. первичный ключ (пункт, дата)], написать запрос с выходными данными (пункт, дата, приход, расход).**
```sql
SELECT i.point, i.date, inc, out
FROM Income_o i
LEFT JOIN Outcome_o o
ON i.point=o.point
   AND i.date=o.date
UNION
SELECT o.point, o.date, inc, out
FROM Income_o i
RIGHT JOIN Outcome_o o
ON i.point=o.point
   AND i.date=o.date
```
****
**Задание: 30 (Serge I: 2003-02-14)
В предположении, что приход и расход денег на каждом пункте приема фиксируется произвольное число раз (первичным ключом в таблицах является столбец code), требуется получить таблицу, в которой каждому пункту за каждую дату выполнения операций будет соответствовать одна строка.
Вывод: point, date, суммарный расход пункта за день (out), суммарный приход пункта за день (inc). Отсутствующие значения считать неопределенными (NULL).**
```sql
SELECT point, date, SUM(out), SUM(inc) 
FROM
(SELECT point, date, null AS out, inc FROM income 
UNION ALL
SELECT point, date, out, null AS inc FROM outcome) x
GROUP BY point, date
```
****
**Рассматривается БД кораблей**, участвовавших во второй мировой войне. Имеются следующие отношения:
* **Classes (class, type, country, numGuns, bore, displacement)**
* **Ships (name, class, launched)**
* **Battles (name, date)**
* **Outcomes (ship, battle, result)**

<br>Корабли в «классах» построены по одному и тому же проекту, и классу присваивается либо имя первого корабля, построенного по данному проекту, либо названию класса дается имя проекта, которое не совпадает ни с одним из кораблей в БД. Корабль, давший название классу, называется головным.

<br>Отношение **Classes** содержит имя класса, тип (bb для боевого (линейного) корабля или bc для боевого крейсера), страну, в которой построен корабль, число главных орудий, калибр орудий (диаметр ствола орудия в дюймах) и водоизмещение ( вес в тоннах).

<br>В отношении **Ships** записаны название корабля, имя его класса и год спуска на воду. 

<br>В отношение **Battles** включены название и дата битвы, в которой участвовали корабли, а в отношении Outcomes – результат участия данного корабля в битве (потоплен-sunk, поврежден - damaged или невредим - OK).
Замечания. 1) 

<br>В отношение **Outcomes** могут входить корабли, отсутствующие в отношении Ships. 2) Потопленный корабль в последующих битвах участия не принимает.
****
**Задание: 32 (Serge I: 2003-02-17)
Одной из характеристик корабля является половина куба калибра его главных орудий (mw). С точностью до 2 десятичных знаков определите среднее значение mw для кораблей каждой страны, у которой есть корабли в базе данных.**
<br>Для начала разведуем наличие головных среди кораблей,
<br>которые есть только в табл. Outcomes
```sql
SELECT country, 
    CAST(AVG(POWER(bore, 3)/2 ) AS DECIMAL(10,2)) 
    AS weight
FROM ((SELECT name, country, bore
FROM classes c JOIN ships s
ON c.class=s.class) 
UNION
(SELECT ship as name, country, bore
FROM classes c JOIN outcomes o
ON c.class=o.ship)) a
GROUP BY country
```
****
**Задание: 35 (qwrqwr: 2012-11-23)
В таблице Product найти модели, которые состоят только из цифр или только из латинских букв (A-Z, без учета регистра).
Вывод: номер модели, тип модели.**
```sql
SELECT model, type
FROM product
WHERE model not LIKE '%[^a-zA-Z]%' or model not LIKE '%[^0-9]%'
```
****
**Задание: 39 (Serge I: 2003-02-14)
Найдите корабли, `сохранившиеся для будущих сражений`; т.е. выведенные из строя в одной битве (damaged), они участвовали в другой, произошедшей позже.**
****

**SQL-ex принимает решение:**
```sql
SELECT DISTINCT a1.ship 
FROM 
(SELECT ship, date, result 
FROM outcomes o 
JOIN battles b 
ON o.battle=b.name
) a1, 
(SELECT ship, date, result 
FROM outcomes o 
JOIN battles b 
ON o.battle=b.name
) a2 
WHERE a1.ship=a2.ship 
AND a1.result='damaged'
AND a1.date<a2.date
```
**Также можно упростить запись представлением:**
```sql
WITH a AS (
    SELECT * 
    FROM outcomes o 
    JOIN battles b 
    ON o.battle=b.name)
    
SELECT DISTINCT a1.ship 
FROM 
a a1, 
a a2 
WHERE a1.ship=a2.ship 
AND a1.result='damaged'
AND a1.date<a2.date
``` 
<br>   Для начала стоит отметить противоречащее само себе условие задачи - 
   сохранившиеся для будущих сражений это просто last battle = OK(last намекает на оконные функции), не важно были ли они когда-либо damaged, но попробуем найти корабли выполняющие оба эти условия
<br>  **Меня насторожило**, что запрос выше допускает ситуацию когда для одного корабля после битвы 1 с результатом damaged, может произойти битва 2 не только с результатом OK, но и с результатом sunk. 
<br>(такой корабль явно не сохраняется для будущих сражений)
<br>  т.е. запросом не исключаются последовательности битв 
<br>**1)damaged 2)sunk**
<br>  Более того, сценарии 
<br>**1)damaged 2)OK last)sunk, 
<br>1)damaged 2)OK last)damaged** 
<br>также не исключаются данным запросом, 
<br>т.к. FROM a1, a2 дает все возможные пары битв(например 1)damaged 2)OK), 
<br>затем WHERE отбирает любые пары битв где result='damaged' в первой из них.
<br>А если корабль утонет после пары битв 1)damaged 2)OK ему все равно.
<br>
<br>**Чтобы исправить это, по-хорошему нужны оконные функции, но попробуем без них
<br>I) отбираю последнюю битву MAX(date) и ее результат + GROUP BY ship в a02
<br>затем self-JOIN по ship и date** (если две битвы в один день, вылезут обе в подзапросе, но если в одной OK, а в другой damaged, мы не знаем какая была посл, по крайней мере после сравнения дат мы не запишем такие неопределенные корабли в результат)
<br>**II) отбираю из a2 только OK**
****
```sql 
WITH a1 AS
(SELECT ship, date, result 
FROM outcomes o 
JOIN battles b 
ON o.battle=b.name)

SELECT DISTINCT a1.ship 
FROM 
a1, 
(SELECT a1.ship, a1.date, result FROM 
(SELECT ship, MAX(date) date
FROM a1
   GROUP BY ship) a02
JOIN a1
ON a02.ship=a1.ship 
AND a02.date=a1.date) a2 
WHERE a1.ship=a2.ship 
AND a1.result='damaged'
AND a1.date<a2.date
```
<br>Стоит добавить в этот запрос AND a2.result='OK' и тогда решение не принимается проверочной базой
<br>sql-ex утверждает что я исключил 5 строк из верного запроса, значит в проверочной базе встречаются 5 строк с сценарием 
<br>**n)damaged   last)!= 'OK', битва n произошла до last**
<br>но верное по мнению sql-ex решение игнорирует эти строки.
****
**Решение оконной**
```sql
WITH a1 AS
(SELECT ship, date, result 
FROM outcomes o 
JOIN battles b 
ON o.battle=b.name)

SELECT DISTINCT c1.ship 
FROM 
a1 c1, 
(SELECT ship, result, date
FROM 
(SELECT ship, result, date,
LAST_VALUE(date) OVER (PARTITION BY ship ORDER BY date RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS last
FROM a1) b1
WHERE date=last) c2 
WHERE c1.ship=c2.ship 
AND c1.result='damaged'
AND c1.date<c2.date
```
<br>Если добавить сюда AND c2.result='OK', то это все также не нравится sql-ex'у
<br> с помощью c2.result!='sunk' и c2.result!='damaged' я выяснил, что 2 из этих 5 недостающих имеют damaged в последних битвах кораблей, а 3 sunk
****
**Задание: 40 (Serge I: 2012-04-20)
Найти производителей, которые выпускают более одной модели, при этом все выпускаемые производителем модели являются продуктами одного типа.
Вывести: maker, type
**
```sql
WITH a AS
(SELECT maker, type
FROM Product)

SELECT DISTINCT *
FROM a
WHERE maker IN
(SELECT maker
FROM a
GROUP BY maker
HAVING COUNT(DISTINCT type)=1 
AND COUNT(type)>1)

```
****
**Задание: 41 (Serge I: 2019-05-31)
Для каждого производителя, у которого присутствуют модели хотя бы в одной из таблиц PC, Laptop или Printer,
определить максимальную цену на его продукцию.
Вывод: имя производителя, если среди цен на продукцию данного производителя присутствует NULL, то выводить для этого производителя NULL,
иначе максимальную цену.**
```sql
WITH a AS(
SELECT *
FROM
((SELECT model, price FROM pc)
UNION
(SELECT model, price FROM laptop)
UNION
(SELECT model, price FROM printer)) c)

SELECT maker, 
(CASE WHEN MIN(CASE WHEN price IS NULL THEN 0 ELSE 1 END) = 1
THEN MAX(price) END) price
FROM product p JOIN a 
ON p.model=a.model
GROUP BY maker
```
****
**Задание: 43 (qwrqwr: 2011-10-28)
Укажите сражения, которые произошли в годы, не совпадающие ни с одним из годов спуска кораблей на воду.**
```sql
SELECT name
FROM battles
WHERE DATEPART(year, date) NOT IN 
(SELECT launched FROM ships
WHERE launched IS NOT NULL)
```
****
**Задание: 45 (Serge I: 2002-12-04)
Найдите названия всех кораблей в базе данных, состоящие из трех и более слов (например, King George V).
Считать, что слова в названиях разделяются единичными пробелами, и нет концевых пробелов**
```sql
SELECT name
FROM
(SELECT name
FROM ships
UNION
SELECT ship as name
FROM outcomes) a
WHERE name LIKE '% % %' 
```
****
**Задание: 46 (Serge I: 2003-02-14)
Для каждого корабля, участвовавшего в сражении при Гвадалканале (Guadalcanal), вывести название, водоизмещение и число орудий.**
```sql
SELECT DISTINCT ship, displacement, numguns
FROM classes c LEFT JOIN ships s
ON c.class=s.class
RIGHT JOIN Outcomes
ON c.class=ship
OR s.name=ship
WHERE battle='Guadalcanal'
```

****
**Задание: 47 (Serge I: 2019-06-07)
Определить страны, которые потеряли в сражениях все свои корабли.**
```sql
WITH sh AS (
SELECT c.country, s.name FROM classes c JOIN ships s ON c.class=s.class
UNION
SELECT c.country, o.ship FROM outcomes o JOIN classes c ON c.class=o.ship
)
, a AS (
SELECT country, name, CASE
   WHEN name NOT IN 
   (SELECT ship AS name FROM outcomes 
   WHERE result='sunk') THEN 0
   ELSE 1
   END AS sunk
FROM sh 
)
SELECT country FROM a
GROUP BY country
HAVING COUNT(DISTINCT name)=SUM(sunk)
```
****
**Задание: 48 (Serge I: 2003-02-16)
Найдите классы кораблей, в которых хотя бы один корабль был потоплен в сражении.**
```sql
WITH sh AS (
SELECT c.class, s.name name FROM classes c JOIN ships s ON c.class=s.class
UNION
SELECT c.class, o.ship name FROM outcomes o JOIN classes c ON c.class=o.ship
)

SELECT DISTINCT class
FROM sh
WHERE name IN (SELECT ship AS name FROM outcomes 
   WHERE result='sunk')

```
****
****
```sql


```
****
****
```sql

```
****
****
```sql


```
****
****
```sql

```
****

